In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load and preprocess the data
user_interaction = pd.read_csv("random_data.csv")
user_interaction['interaction_score'] = user_interaction['likes'] * 3 + user_interaction['comments'] * 1 + user_interaction['save'] * 2
user_interaction['normalized_watch_time'] = user_interaction['watch_time_seconds'] / user_interaction['watch_time_seconds'].max()
user_interaction['interaction_score'] += user_interaction['normalized_watch_time']
interaction_data = user_interaction[['user_id', 'video_id', 'interaction_score']]

In [6]:
# Encode user_id and video_id as categorical variables
interaction_data_encoded = interaction_data.copy()
interaction_data_encoded['user_id'] = interaction_data_encoded['user_id'].astype('category').cat.codes.values
interaction_data_encoded['video_id'] = interaction_data_encoded['video_id'].astype('category').cat.codes.values

In [7]:
# Split the data into training and testing sets
train, test = train_test_split(interaction_data_encoded, test_size=0.2, random_state=42)

In [8]:
# Scale the interaction_score to the range [0, 1]
scaler = MinMaxScaler()
train['interaction_score'] = scaler.fit_transform(train[['interaction_score']])
test['interaction_score'] = scaler.transform(test[['interaction_score']])

In [9]:
# Define the neural network model
user_input = Input(shape=(1,), name='user_input')
user_embedding = Embedding(input_dim=len(interaction_data['user_id'].unique()), output_dim=50, name='user_embedding')(user_input)
user_flatten = Flatten(name='user_flatten')(user_embedding)

video_input = Input(shape=(1,), name='video_input')
video_embedding = Embedding(input_dim=len(interaction_data['video_id'].unique()), output_dim=50, name='video_embedding')(video_input)
video_flatten = Flatten(name='video_flatten')(video_embedding)

concat = Concatenate(name='concat')([user_flatten, video_flatten])
dense = Dense(128, activation='relu', name='dense')(concat)
dropout = Dropout(0.5)(dense)
output = Dense(1, activation='sigmoid', name='output')(dense)

model = Model(inputs=[user_input, video_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [10]:
# Train the model
model.fit([train['user_id'], train['video_id']], train['interaction_score'], epochs=50, batch_size=64, validation_split=0.2)

Epoch 1/50
100/100 [==============================] - 1s 2ms/step - loss: 0.0745 - val_loss: 0.0760
Epoch 2/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0725 - val_loss: 0.0778
Epoch 3/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0700 - val_loss: 0.0790
Epoch 4/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0668 - val_loss: 0.0802
Epoch 5/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0627 - val_loss: 0.0829
Epoch 6/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0570 - val_loss: 0.0840
Epoch 7/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0508 - val_loss: 0.0900
Epoch 8/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0444 - val_loss: 0.0932
Epoch 9/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0391 - val_loss: 0.0991
Epoch 10/50
100/100 [==============================] - 0s 1ms/step - loss: 0.0342 - val_loss: 0.1009

In [11]:
# Save the model to an .h5 file
model.save('video_recommender.h5')

In [12]:
# Load the model
loaded_model = tf.keras.models.load_model('video_recommender.h5')

In [13]:
def get_top_20_recommendations(user_id, model, interaction_data):
    video_ids = interaction_data['video_id'].unique()
    user_ids = np.array([user_id] * len(video_ids))
    predictions = model.predict([user_ids, video_ids])
    sorted_indices = np.argsort(predictions.flatten())[::-1]
    return video_ids[sorted_indices[:20]].flatten().tolist()

user_id = 10099
user_num = user_id-10000
top_20_recommendations = get_top_20_recommendations(user_num, loaded_model, interaction_data_encoded)
print(top_20_recommendations)

16/16 [==============================] - 0s 968us/step
[359, 301, 454, 163, 411, 154, 236, 193, 363, 216, 409, 157, 475, 365, 49, 206, 147, 317, 491, 261]
